### Goals  
    1. Load the clinical variables
    2. Process the data as a pandas dataframe
    3. Research SurvhShap, speficially what kind of data it expects.
    4. Create m parameter-cuts of the data as equidistant
    5. Implement an event-sorting algorithm based on the cuts and events.
    6. Finish processing the data so its ready for training
    7. Setup a network alongside the binary-cross-loss (sigmoid on the network output).
    8. Implement a function to create survival function for the chosen slices (0, m)
    9. Plot survival for five individuals
    10. plot accuracy
    11. attempt C-index or Brier score on the network outputs.